In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [2]:
#naver map api key
client_id = '';    # 본인이 할당받은 ID 입력
client_pw = '';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='


In [4]:

# 주소 목록 파일 (.csv)
data = pd.read_csv('pop_fin.csv',encoding='utf8')
# 주소 목록 파일 (.xlsx)
# data = pd.read_csv('C:/Users/admin/Desktop/dlrl.csv',encoding='ansi')

In [21]:
data2 = pd.read_csv('seoul_pop_g3b4.csv',encoding='utf8')

In [22]:
data2 = data2.iloc[:2,]

In [25]:
data2['road'] = data2['seoul']+' '+data2['gu']+' '+data2['도로명']

In [26]:
data2

,year,seoul,gu,emd,pop_total,도로명,도로명_합,pop_average,road
0,2020,서울특별시,강남구,청담동,3909208,학동로77길,서울특별시강남구학동로77길,4030935.819,서울특별시 강남구 학동로77길
1,2020,서울특별시,강남구,논현1동,4204012,학동로6길,서울특별시강남구학동로6길,4030935.819,서울특별시 강남구 학동로6길


In [31]:
data3 = pd.read_csv('./seoul_popssss.csv',encoding='ansi')

In [35]:
data4=data3.iloc[:,:7]

In [37]:
data4.drop(columns=['기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드'],inplace=True)

In [39]:
data4.rename(columns={'기준_년_코드':'year','상권_코드_명':'road','총_생활인구_수':'pop'},inplace=True)

In [46]:
data4

,year,road,pop
0,2021,풍성로37가길,9759687
1,2021,천호옛길,8699419
2,2021,천호옛14길,7252275
3,2021,천호대로219길,1457777
4,2021,천호대로197길,5802224
...,...,...,...
13125,2018,백석동길,52333
13126,2018,명륜길,1400238
13127,2018,돈화문로11가길,1747636
13128,2018,난계로27길,2304441


In [42]:
data5 = data4.iloc[:2,]

In [45]:
data5

,year,road,pop
0,2021,풍성로37가길,9759687
1,2021,천호옛길,8699419


In [44]:
geo_coordi = []     

# for add in ['서울역','역삼역']:
for add in data5['road']:
#     add2 = '서울특별시 '+add
    add_urlenc = parse.quote(add)
#     add_urlenc = add
    url = api_url + add_urlenc
    print(url)
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print(response_body)
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])

print(geo_coordi)


https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%ED%92%8D%EC%84%B1%EB%A1%9C37%EA%B0%80%EA%B8%B8
'result' not exist!
https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%B2%9C%ED%98%B8%EC%98%9B%EA%B8%B8
'result' not exist!
[[None, None], [None, None]]


In [7]:
geo_coordi

[['37.5196312', '127.0187718'],
 ['37.5130739', '127.0117668'],
 ['37.5048477', '127.0048951'],
 ['37.4948912', '127.0173808'],
 ['37.4866487', '127.0153635'],
 ['37.4870004', '127.0466934'],
 ['37.5325585', '126.8462563'],
 ['37.5724334', '126.9769020'],
 ['37.5714809', '126.9918969'],
 ['37.5665336', '126.9965401'],
 ['37.5646558', '127.0056069'],
 ['37.5603611', '127.0138788'],
 ['37.5624304', '127.0603233'],
 ['37.5403541', '127.1258332'],
 ['37.4923606', '127.0307188'],
 ['37.4905423', '127.0089712'],
 ['37.4812897', '126.9970307'],
 ['37.5372254', '127.1348042'],
 ['37.5460517', '127.1432073'],
 ['37.5512570', '127.1440039'],
 ['37.5556292', '127.1538847'],
 ['37.5535023', '127.1569018'],
 ['37.5165606', '127.1311459'],
 ['37.5080279', '127.1254567'],
 ['37.5034262', '127.1261696'],
 ['37.4985772', '127.1338152'],
 ['37.4942534', '127.1420291'],
 ['37.4944609', '127.1519774'],
 [None, None],
 ['37.5870736', '127.2086664'],
 ['37.5794275', '127.2226848'],
 ['37.5468290', '127.2438

In [9]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({'subway_st'
                            "gibun": data['지번주소'].values,
                              "lng": np_geo_coordi[:, 0],
                              "lat": np_geo_coordi[:, 1]})

pd_geo_coordi.to_csv('./subway_point.csv', encoding=ding='ansi')